In [ ]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2016, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


In [ ]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "pdfs6"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', "2016-6-1", 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification Updates")
#os.startfile('"%s"'%res)
print()
print("Done")

# 3/2/2017 and 4/5/2017

In [ ]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "Software_FRS_170405"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

# 5/4/2017

In [2]:
from scripts.tools.issuetracker.frs_update import main
import shutil
import datetime

def bt():
    return datetime.date.today().strftime("%y%m%d")
    
pdfpth = "Software_FRS_" + bt()

try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038C2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

KeyboardInterrupt: 